In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import tikzplotlib
import re

try:
    sys.path.append(str(Path().cwd().parent))
except IndexError:
    pass

from vibromaf.util import matlab
from vibromaf.util import model
from vibromaf.util.common import *
from vibromaf import config

In [ ]:
FIGURE_PATH = config.PROJECT_ROOT_PATH / "paper" / "figures"

In [ ]:
test_indices = [
    # list(range(8))  # evaluate for all signals
    0,  # Aluminum Grid - fast
    5,  # Polyester Pad - slower
]

SNR = matlab.load_data_for_metric("SNR", test_indices=test_indices)[1]
SIM = matlab.load_data_for_metric("SIM", test_indices=test_indices)[1]
SPQI = matlab.load_data_for_metric("SPQI", test_indices=test_indices)[1]

vibromaf_input = np.array([SNR, SIM, SPQI]).transpose()

reference = matlab.load_data_for_metric("InterNormRating", test_indices=test_indices)[1]

In [ ]:
regressor = model.load_model(config.MODEL_PATH / "vibromaf-v1.pickle")

In [ ]:
prediction = regressor.predict(vibromaf_input)
print_mse_and_pc("vibromaf", reference / 10, prediction / 10)

In [ ]:
reference_per_codec = matlab.split_per_codec(reference)
vibromaf_per_codec = matlab.split_per_codec(prediction)

spqi_per_codec = matlab.split_per_codec(SPQI)
stsim_per_codec = matlab.split_per_codec(SIM)

In [ ]:
vcpwq_ref = np.flip(
    np.mean(matlab.reshape_per_compression_rate(reference_per_codec[0]), axis=1) / 10
)
vcpwq_ref_std = np.flip(
    np.std(matlab.reshape_per_compression_rate(reference_per_codec[0]), axis=1) / 10
)
vcpwq_vibromaf = np.flip(
    np.mean(matlab.reshape_per_compression_rate(vibromaf_per_codec[0]), axis=1) / 10
)

pvcslp_ref = np.flip(
    np.mean(matlab.reshape_per_compression_rate(reference_per_codec[1]), axis=1) / 10
)
pvcslp_ref_std = np.flip(
    np.std(matlab.reshape_per_compression_rate(reference_per_codec[1]), axis=1) / 10
)
pvcslp_vibromaf = np.flip(
    np.mean(matlab.reshape_per_compression_rate(vibromaf_per_codec[1]), axis=1) / 10
)

vpcds_ref = np.flip(
    np.mean(matlab.reshape_per_compression_rate(reference_per_codec[2]), axis=1) / 10
)
vpcds_ref_std = np.flip(
    np.std(matlab.reshape_per_compression_rate(reference_per_codec[2]), axis=1) / 10
)
vpcds_vibromaf = np.flip(
    np.mean(matlab.reshape_per_compression_rate(vibromaf_per_codec[2]), axis=1) / 10
)

In [ ]:
cr_vcpwq = np.flip(
    np.mean(
        matlab.load_signal_from_mat(config.DATA_PATH / "CR_VCPWQ.mat", "CR_VCPWQ"),
        axis=1,
    )
)
cr_pvcslp = np.flip(
    np.mean(
        matlab.load_signal_from_mat(config.DATA_PATH / "CR_PVCSLP.mat", "CR_PVCSLP"),
        axis=1,
    )
)
cr_vpcds = np.flip(
    np.mean(
        matlab.load_signal_from_mat(config.DATA_PATH / "CR_VPCDS.mat", "CR_VPCDS"),
        axis=1,
    )
)

In [ ]:
plt.plot(cr_vcpwq, vcpwq_ref, "b-", label="VC-PWQ, Rating")
plt.plot(cr_vcpwq, vcpwq_vibromaf, "b--", label="VC-PWQ, VibroMAF")

plt.plot(cr_pvcslp, pvcslp_ref, "g-", label="PVC-SLP, Rating")
plt.plot(cr_pvcslp, pvcslp_vibromaf, "g--", label="PVC-SLP, VibroMAF")

plt.plot(cr_vpcds, vpcds_ref, "r-", label="VPC-DS, Rating")
plt.plot(cr_vpcds, vpcds_vibromaf, "r--", label="VPC-DS, VibroMAF")

plt.xlim(0, 50)
plt.ylim(0, 1)
plt.xlabel("Compression Ratio")
plt.ylabel("VibroMAF Score")
plt.legend()
plt.grid()

tikzplotlib.save(FIGURE_PATH / "vibromaf-per-codec.tex")

In [ ]:
vcpwq_sqpi = np.flip(
    np.mean(matlab.reshape_per_compression_rate(spqi_per_codec[0]), axis=1)
)
vcpwq_stsim = np.flip(
    np.mean(matlab.reshape_per_compression_rate(stsim_per_codec[0]), axis=1)
)

pvcslp_sqpi = np.flip(
    np.mean(matlab.reshape_per_compression_rate(spqi_per_codec[1]), axis=1)
)
pvcslp_stsim = np.flip(
    np.mean(matlab.reshape_per_compression_rate(stsim_per_codec[1]), axis=1)
)

vpcds_sqpi = np.flip(
    np.mean(matlab.reshape_per_compression_rate(spqi_per_codec[2]), axis=1)
)
vpcds_stsim = np.flip(
    np.mean(matlab.reshape_per_compression_rate(stsim_per_codec[2]), axis=1)
)

In [ ]:
plt.plot(cr_vcpwq, vcpwq_ref, "b-", label="VC-PWQ, Rating")
plt.plot(cr_vcpwq, vcpwq_sqpi, "b--", label="VC-PWQ, SPQI")
plt.plot(cr_vcpwq, vcpwq_stsim, "b-.", label="VC-PWQ, ST-SIM")

plt.plot(cr_pvcslp, pvcslp_ref, "g-", label="PVC-SLP, Rating")
plt.plot(cr_pvcslp, pvcslp_sqpi, "g--", label="PVC-SLP, SPQI")
plt.plot(cr_pvcslp, pvcslp_stsim, "g-.", label="PVC-SLP, ST-SIM")

plt.plot(cr_vpcds, vpcds_ref, "r-", label="VPC-DS, Rating")
plt.plot(cr_vpcds, vpcds_sqpi, "r--", label="VPC-DS, SPQI")
plt.plot(cr_vpcds, vpcds_stsim, "r-.", label="VPC-DS, ST-SIM")

plt.xlim(0, 50)
plt.ylim(0, 1)
plt.xlabel("Compression Ratio")
plt.ylabel("Score")
plt.legend()
plt.grid()

tikzplotlib.save(FIGURE_PATH / "metric-per-codec.tex")

In [ ]:
plt.errorbar(
    cr_vcpwq, vcpwq_ref, yerr=vcpwq_ref_std, fmt="b-", capsize=3, label="VC-PWQ"
)
plt.fill_between(
    cr_vcpwq,
    vcpwq_ref - vcpwq_ref_std,
    vcpwq_ref + vcpwq_ref_std,
    facecolor="b",
    alpha=0.3,
)
plt.errorbar(
    cr_pvcslp, pvcslp_ref, yerr=pvcslp_ref_std, fmt="g-", capsize=3, label="PVC-SLP"
)
plt.fill_between(
    cr_pvcslp,
    pvcslp_ref - pvcslp_ref_std,
    pvcslp_ref + pvcslp_ref_std,
    facecolor="g",
    alpha=0.3,
)
plt.errorbar(
    cr_vpcds, vpcds_ref, yerr=vpcds_ref_std, fmt="r-", capsize=3, label="VPC-DS"
)
plt.fill_between(
    cr_vpcds,
    vpcds_ref - vpcds_ref_std,
    vpcds_ref + vpcds_ref_std,
    facecolor="r",
    alpha=0.3,
)

plt.xlim(0, 50)
plt.ylim(0, 1)
plt.xlabel("Compression Ratio")
plt.ylabel("Score")
plt.legend()
plt.grid()

tikzplotlib.save(FIGURE_PATH / "rating-per-codec.tex")

In [ ]:
def insert_header_template(file: Path):
    text = file.read_text()
    result = re.sub(
        r"\\begin{axis}\[\n(.[^\]]*)\]", r"\\input{figures/plot-header.tex}", text
    )
    file.write_text(result)


insert_header_template(FIGURE_PATH / "vibromaf-per-codec.tex")
insert_header_template(FIGURE_PATH / "metric-per-codec.tex")
insert_header_template(FIGURE_PATH / "rating-per-codec.tex")